In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils.np_utils as ku 
import numpy as np 
import pandas as pd
from nltk import word_tokenize, ngrams
import contractions
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

In [2]:
def create_model(predictors, label, max_sequence_len, total_words, num_epochs):
    
    model = Sequential()
    model.add(Embedding(total_words, 32, input_length=max_sequence_len-1))
    model.add(LSTM(150, return_sequences = True))
    # model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    model.fit(predictors, label, epochs=num_epochs, verbose=1, callbacks=[earlystop])
    print(model.summary())
    return model 

In [3]:
def csv_to_lst(file):
    """
    Grabs the lyric section of the given csv and convert it to a string
    Parameters:
        file (CSV): file path where csv is located
    Returns:
        A list string representing all of the text section in the csv
    """
    lyrics_df = pd.read_csv(file) 
#     lyrics_df = lyrics_df

    lyrics_list = list(lyrics_df['lyrics'])


    return lyrics_list

print('done')

done


In [4]:
def pre_process_all_lyrics(lyric_list):
    processed = []
    for lyrics in lyric_list:
#         print(lyrics)
        processed.append(pre_process_text(lyrics))
    return processed

In [5]:
def pre_process_line(line):
    EMBED = 'embed'
    word_list = word_tokenize(line)
    
   
    #index = word_list.index('Lyrics')
    #word_list = word_list[index + 1:]
    
    no_brackets_list = []
    is_inside = False
    punctuation_list = '''!()-[];:'"\,<>./?@#$%^&*_~”“’‘--...``'''''
    
    for word in word_list:
        if word == '[':
            is_inside = True
        if word == ']':
            is_inside = False
        else:
            if not is_inside:
                if word not in punctuation_list:
                    no_brackets_list.append(word.lower())

    if len(no_brackets_list) > 0:
        end_word = no_brackets_list[len(no_brackets_list)-1]   
        if EMBED in end_word:
#             print(end_word)
            embed_location = end_word.index('embed') 
            # all lyric genius data comes with word 'Embed' at the end
            substr = end_word[:embed_location]
            if len(substr) > 0:
                no_brackets_list[len(no_brackets_list)-1] = substr
        
        end_word = no_brackets_list[len(no_brackets_list)-1]  
        new_end_word = ''
        # also sometimes has a number before embed in last word
        for c in end_word:
            if c.isdigit():
                break
            else:
                new_end_word = new_end_word + c
        if len(end_word) > 0:
            no_brackets_list[len(no_brackets_list)-1] = new_end_word
            
    ret = []
    for word in no_brackets_list:
        if len(word) < 0:
            break
        #idk why this is like this 
        elif word == "''":
            break
        else:
            ret.append(word)
    return ret

In [6]:
def pre_process_text(lyrics):
    """
    Preprocesses the text to add start and end tokens to each sentence
    Parameters:
        lst (list): list of lyrics 
    Returns:
        List of List of words with start and end tokens
    """
    
    lyrics = contractions.fix(lyrics)
#     print(lyrics)
    line_list = lyrics.split("\n")
    line_list = line_list[1:]
    
    ret = []
    for line in line_list:
        if not len(line) == 0:
            to_be_appended = pre_process_line(line)
            if len(to_be_appended) > 0:
                ret.append(to_be_appended)
    return ret

In [7]:
def dataset_preparation(data):

    # basic cleanup
#     corpus = data.lower().split("\n")
    # tokenization	
    corpus = flatten_lines(data)
    print(corpus[:10])
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    print('tokenization:')
    print(tokenizer.word_index)
    # create input sequences using list of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    print('input_sequences:')
    print(input_sequences)
    # pad sequences 
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    print('padded input_sequences:')
    print(input_sequences)
    # create predictors and label
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    
    print('X predictors:')
    print(predictors)
    print('y labels:')
    print(label)
    label = ku.to_categorical(label, num_classes=total_words)
    
    return predictors, label, max_sequence_len, total_words

In [8]:
def flatten_lines(data):
    corpus = []
    for song in data:
        for line in song:
            line_string = ""
            for word in line: 
                line_string = line_string + " " + word
            corpus.append(line_string)
    return corpus

In [9]:
lst = csv_to_lst('taylorswift.csv')
data = pre_process_all_lyrics(lst)[:10]
# print(data)

In [10]:
# for song in data:
#     for line in song:
#         print(line)

In [11]:
predictors, label, max_sequence_len, total_words = dataset_preparation(data)

[' i walked through the door with you the air was cold', " but something 'bout it felt like home somehow", " and i left my scarf there at your sister 's house", ' and you have still got it in your drawer even now', ' oh your sweet disposition and my wide-eyed gaze', ' we are singin in the car getting lost upstate', ' autumn leaves fallin down like pieces into place', ' and i can picture it after all these days', ' and i know it is long gone and', " that magic 's not here no more"]
tokenization:
{'i': 1, 'you': 2, 'the': 3, 'it': 4, 'and': 5, 'me': 6, 'was': 7, 'to': 8, 'my': 9, 'all': 10, 'in': 11, 'your': 12, 'do': 13, 'there': 14, 'a': 15, 'not': 16, 'but': 17, 'what': 18, 'like': 19, 'of': 20, 'is': 21, 'be': 22, 'just': 23, 'made': 24, 'we': 25, 'would': 26, 'remember': 27, 'want': 28, 'never': 29, 'on': 30, 'look': 31, 'have': 32, 'am': 33, 'well': 34, 'for': 35, 'too': 36, 'are': 37, 'because': 38, 'when': 39, 'were': 40, 'know': 41, 'so': 42, 'this': 43, 'time': 44, 'can': 45, '

In [12]:
print('predictors: ', len(predictors))
print(predictors[:3])
print('labels: ', label.shape)
print(label[:3])
print('max sequence len: ', max_sequence_len)
print('total words: ', total_words)

predictors:  5007
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1 180]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1 180 293]]
labels:  (5007, 813)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
max sequence len:  18
total words:  813


In [ ]:
model = create_model(predictors, label, max_sequence_len, total_words, 100)

2022-04-29 16:28:49.943893: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
157/157 [==============================] - 7s 26ms/step - loss: 5.7224 - accuracy: 0.0515
Epoch 2/100
157/157 [==============================] - 4s 22ms/step - loss: 5.4323 - accuracy: 0.0519
Epoch 3/100
157/157 [==============================] - 3s 22ms/step - loss: 5.2516 - accuracy: 0.0537
Epoch 4/100
157/157 [==============================] - 4s 25ms/step - loss: 5.0661 - accuracy: 0.0559
Epoch 5/100
157/157 [==============================] - 4s 22ms/step - loss: 4.8763 - accuracy: 0.0791
Epoch 6/100
157/157 [==============================] - 4s 24ms/step - loss: 4.7109 - accuracy: 0.1023
Epoch 7/100
157/157 [==============================] - 4s 22ms/step - loss: 4.5613 - accuracy: 0.1124
Epoch 8/100
157/157 [==============================] - 4s 23ms/step - loss: 4.4087 - accuracy: 0.1322
Epoch 9/100
157/157 [==============================] - 4s 24ms/step - loss: 4.2692 - accuracy: 0.1420
Epoch 10/100
157/157 [==============================] - 5s 31ms/step - loss: 4.155

157/157 [==============================] - 5s 29ms/step - loss: 2.6339 - accuracy: 0.3970
Epoch 28/100
157/157 [==============================] - 5s 30ms/step - loss: 2.5686 - accuracy: 0.4154
Epoch 29/100
157/157 [==============================] - 6s 37ms/step - loss: 2.5082 - accuracy: 0.4282
Epoch 30/100
157/157 [==============================] - 5s 31ms/step - loss: 2.4452 - accuracy: 0.4392
Epoch 31/100
157/157 [==============================] - 5s 33ms/step - loss: 2.3852 - accuracy: 0.4596
Epoch 32/100
 62/157 [==========>...................] - ETA: 3s - loss: 2.3681 - accuracy: 0.4486

In [ ]:
def generate_text(tokenizer, seed_text, next_words, max_sequence_len):
#     lines = []
    LINE_LEN = np.random.randint(1, max_sequence_len)
    words = []
    words.extend(seed_text.split())
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
#         print('-----')
#         print(predicted.shape)
        choice = np.argmax(predicted)
#         choice = np.random.choice(len(predicted[0]), p=predicted[0])
#         choice = sample(predicted[0])
#         print('choice: ', choice)
#         print(index_of)
        word = tokenizer.index_word[choice]
        print('word: ', word)
        if (len(words) % LINE_LEN == 0): # new line time
#             print(words)
#             print(len(words))
            words.append('\n')
            seed_text = word
        else:
            words.append(word)
            seed_text = seed_text + " " + word
#         seed_text = word
#         output_word = ""
#         for word, index in tokenizer.word_index.items():
#             if index == predicted[0]:
#                 output_word = word
#                 break
#         seed_text += " " + output_word
    return words

In [ ]:
def sample(preds, temperature=0.5):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def format_lyrics(words):
    lyr = ""
    for word in words:
        lyr = lyr + " " + word
    return lyr

In [ ]:
generated = generate_text(tokenizer, "you", 15, max_sequence_len)
lyrics = format_lyrics(generated)
print('final lyrics:')
print(lyrics)

In [ ]:
generated2 = generate_text(tokenizer, "mom", 20, max_sequence_len)
print(generated2)

In [ ]:
generated3 = generate_text(tokenizer, "this is", 20, max_sequence_len)
print(generated3)

In [ ]:
generated4 = generate_text(tokenizer, "his", 20, max_sequence_len)
print(generated4)